<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Plotting-central-path---of-an-LP" data-toc-modified-id="Plotting-central-path---of-an-LP-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Plotting central path - of an LP</a></span></li><li><span><a href="#Solving-by-IPM" data-toc-modified-id="Solving-by-IPM-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Solving by IPM</a></span></li></ul></div>

In [ ]:
# uncomment and run once
# Using Pkg
#Pkg.add("Plots")
#Pkg.add("LinearAlgebra")
#Pkg.add("OPtim")


In [ ]:
#Pkg.add("PlotlyJS")
#Pkg.add("ORCA")# if you want to try plotlyjs()

In [ ]:
using Plots, LinearAlgebra, Optim 
plotlyjs() #Plots has multiple plotting backend. plotlyjs() has more interactivity in notebook but sometimes buggy
#pyplot()

In [ ]:
f = x -> (1-x[1])^2 + 5*(x[2] - x[1]^2)^2
df = x -> [2*(10*x[1]^3-10*x[1]*x[2]+x[1]-1), 10*(x[2]-x[1]^2)]
hf = x -> [60x[1]^2-20x[2]+2 -20x[1]; -20x[1] 10]

xdomain = -2:0.01:2
ydomain = -2:0.01:2

lev_line = contour(xdomain,ydomain,(x,y)->f([x,y]), levels = 200);
lev_line = plot!(lev_line,[1],[1],seriestype = :scatter, color = :red,label="solution");
lev_line

In [ ]:
x0 = Float64[-0.5, 2]
α0 = 0.02
IT_MAX = 5000
ε = 1e-6

function alg_grad(x0,f,df;α0=α0)
    x = x0
    
     pts = [x]
    vals = [f(x)]
    grads = [df(x)]

    stop_test = false
    it = 0 #iteration number

    while !stop_test
        g = - df(x)       # descent direction
        α = α0            # step size
        x += α*g          # step

        # keeping track of results
        push!(pts,x)
        push!(vals,f(x))
        push!(grads,g)
        
        # stopping test
        it += 1
        stop_test = (it > IT_MAX - 1) || (norm(g)<ε)
    end
    
    println("gradient algorithm with fixed step α=",α0, " ended in ", it, " iterations")
    return pts, vals, grads
   
end

pts, vals, grads = alg_grad(x0,f,df,α0=0.02);

In [ ]:
function plot_trajectory(pts;color="blue",label=`AUTO`,plt=lev_line)
    plt = deepcopy(plt)
    plot!(plt,[p[1] for p in pts],[p[2] for p in pts],color=color,label=label,shape=:cross)
    return plt
end

plt = plot_trajectory(pts,label="gradient")

In [ ]:
# a very simple linear search guaranteeing that value is indeed decreasing
# if you set it_max = 1 you only keep the step = 1. 
# You can do this to explore why it is not always a good idea
function reducing_step(f,x,d;reducing_factor=.9,it_max = 10, verbose = true)
    f0 = f(x)
    α = 1
    it = 0
    while (f(x+α*d) > f(x) + 1e-8) && (it <= it_max)
        α = α*reducing_factor
        it += 1
        verbose && println("step reduced, it = ", it) 
    end
    return α
end

In [ ]:
function alg_Newton(x0,f,df, hf;reduced_step=false)
    x = x0
    
    pts = [x]
    vals = [f(x)]
    grads = [df(x)]

    stop_test = false
    it = 0 #iteration number

    while !stop_test
        g = df(x)         # gradient
        H = hf(x)         # hessian
        d = - H \ g       # descent direction 
        reduced_step ? α=reducing_step(f,x,d) : α = 1   # step size
        x += α*d          # step

        # keeping track of results
        push!(pts,x)
        push!(vals,f(x))
        push!(grads,g)
        
        # stopping test
        it += 1
        stop_test = (it > IT_MAX - 1) || (norm(g)<ε)
    end
    
    println("Newton algorithm ended in ", it, " iterations")
    return pts, vals, grads
   
end

pts,vals,grads = alg_Newton(x0,f,df, hf)
pts2,vals2,grads2 = alg_Newton(x0,f,df, hf,reduced_step=true)

plt=plot_trajectory(pts,label="Newton",color="green",plt = plt)
plt=plot_trajectory(pts2,label="Newton_reduced",color="violet",plt = plt)

## Plotting central path - of an LP

In [ ]:
using Optim, LinearAlgebra, Plots

# constructing the LP min {c'x | Ax <= b}
A = [-1 0; 0 -1 ; 0.5 1; 1 -1]
b = [0 ; 0 ; 1; 0.5]

c = [0.1, 1]


m,n = size(A)

function ln(x) #extended ln to avoid undefinite value
    x >= 10^(-6) ? (return log(x)) : (return -10 + 10^8*x)
end

In [ ]:
# Logarithmic penalty
ϕ(x) =  - sum([ln(b[i]-A[i,:]'x) for i in 1:m])

function gradient_phi(x)
    grad = zeros(n)
    # A REMPLIR 
    for i in 1:m
        grad += 1/(b[i]-A[i,:]'x)*A[i,:]
    end
    return grad
end
gradient_phi(x0)

function finite_diff(x,δ)
  println((ϕ(x + [δ, 0]) - ϕ(x - [δ, 0]))/2δ)
  println((ϕ(x + [0, δ]) - ϕ(x - [0, δ]))/2δ)
end

finite_diff(x0,0.001) # checking gradient by finite difference
gradient_phi(x0)

In [ ]:
function central_point(t,c)
    f(x) = t*c'*x  + ϕ(x)
    x0 = [0.4,0.4]
    res = optimize(f, x0)
    return Optim.minimizer(res) # using an external optimization library
end

function central_path(t_list,c)
    path = []
    for t in t_list
        x = central_point(t,c)
        push!(path,x)
        #println(x[1],' ',x[2])
    end
    return path
end

In [ ]:
#plotting tool
function plot_trajectory(pts;color="blue",label="")
    plt = plot(Shape([0, 0.5, 1,0],[0, 0., 0.5,1]),opacity=.5,label="")
    plot!(plt,[p[1] for p in pts],[p[2] for p in pts],color=color,label=label,shape=:cross)
    return plt
end

In [ ]:
# plotting the central path
t0 = 1

c_path = central_path([t0*1.1^t for t in 0:50],c)
plot_trajectory(c_path)

## Solving by IPM

In [ ]:
function Newton_direction(x,t,c) # compute Newton direction from x for t*c'x+ϕ(x)
    # A REMPLIR
    d = [1/(b[i]-A[i,:]'x) for i in 1:m]
    G = t*c + A'd 
    H = A'Diagonal(d)^2 *A
    return - H \ G
end

function Newton(x,t,c,n=5)
    f(x) = t*c'x + ϕ(x)
    for _ in 1:n
        d = Newton_direction(x,t,c)
        α = reducing_step(f,x,d)
        x += α*Newton_direction(x,t,c) 
        #println("(",x[1],",",x[2],")")
    end
    return x
end

# Checking that Newton's algorithm is converging
Newton(x0,t0,c,10) - central_point(t0,c)

In [ ]:
function IPM(x0,t0,ρ,c; N_outer = 5, N_inner = 3,compute_central_points=false)
    xout_list = [x0] # list of initial points of each outer iteration
    x_list =[x0] # list of all the points
    cp_list =[] # list of the central points associated to t for each outer iteration
    x = x0
    t = t0
    
    # A REMPLIR
    for n_outer in 1:N_outer
        f(x) = t*c'x + ϕ(x)
        for n_inner in 1:N_inner
            d = Newton_direction(x,t,c)
            α = reducing_step(f,x,d)
            x += α*Newton_direction(x,t,c)
            push!(x_list,x)
        end
        push!(xout_list,x)
        compute_central_points && push!(cp_list,central_point(t,c))
        t = ρ*t
         
    end
    
    # if compute_central_points is true return 3 list otherwise 2 
    compute_central_points ? (return x_list,xout_list,cp_list) : (return x_list,xout_list)
    
end

x_list,xout_list = IPM(x0,t0,ρ,c; N_outer = 5, N_inner = 3)
x_list

In [ ]:
c = [0.1,1] # cost
x0 = [0.5,0.5]

#### Algorithm parameters
ρ = 2. # multiplier of t between 
t0 = 1 # initial t

N_outer = 10 # number of outer iterations
N_inner = 3  # number of inner iterations

# recompute central path if you change c = [0.1,1] 
#c_path = central_path([t0*1.1^t for t in 0:50],c)



function plot_IPM(x0,t0,ρ,c; N_outer = 5, N_inner = 5,plot_central_points=true)
    
    #compute trajectory
    if plot_central_points
        x_list,xout_list, cp_list = IPM(x0,t0,ρ,c, N_outer = N_outer, N_inner = N_inner,compute_central_points=true)
    else
        x_list,xout_list = IPM(x0,t0,ρ,c, N_outer = N_outer, N_inner = N_inner)
    end
    
    # plot polyhedron
    plt = plot(Shape([0, 0.5, 1,0],[0, 0., 0.5,1]),opacity=.5,label="",size=(600,400))
    
    # plot -c (scaled)
    plot!(plt,[x0[1],x0[1]-0.1*c[1]],[x0[2],x0[2]-0.1*c[2]],arrow=2, label="-c",lw=2)
    
    #plot central path
    plot!(plt,[p[1] for p in c_path],[p[2] for p in c_path],color="blue",label="central path",lw=2)
    
    # plot IPM trajectory
    plot!(plt,[p[1] for p in x_list],[p[2] for p in x_list],color="orange",label="IPM",shape=:cross,lw=2)
    # overline outer points
    plot!(plt,[p[1] for p in xout_list],[p[2] for p in xout_list],
        seriestype = :scatter,color="red", markersize=6,label="",shape=:cross)
    
    #plot central points
    if plot_central_points
        plot!(plt,[p[1] for p in cp_list],[p[2] for p in cp_list],
            seriestype = :scatter,color="violet", markersize=4,label="central points",shape=:diamond)
    end
    return plt
end

plt = plot_IPM(x0,t0,ρ,c, N_outer =N_outer, N_inner = N_inner);
gui(plt)